# optuna benchmark data

In [1]:
import sys
sys.path.append("../codes")

import numpy as np
import pandas as pd
import torch
import torch.optim as optim
import optuna


from functools import partial
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset

from sklearn.model_selection import KFold
from torch.utils.data.dataset import Subset

from optuna_cv_utils import cv_train, cv_test, make_datasets
from set_data_folder import make_train_data

from model import GCN
from DGCNN import DGCNN_Model

In [2]:
def get_optimizer(trial, model):
    # optimizer をAdamとMomentum SGDで探索
    optimizer_names = ['Adam', "Adagrad"]
    optimizer_name = trial.suggest_categorical('optimizer', optimizer_names)

    # weight decayの探索
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-10, 1e-3)

    # optimizer_nameで分岐
    if optimizer_name == optimizer_names[0]: 
        adam_lr = trial.suggest_loguniform('adam_lr', 1e-5, 1e-1)
        optimizer = optim.Adam(model.parameters(), lr=adam_lr, weight_decay=weight_decay)
    elif optimizer_name == optimizer_names[1]:
        adam_lr = trial.suggest_loguniform('adam_lr', 1e-5, 1e-1)
        optimizer = optim.Adagrad(model.parameters(), lr=adam_lr, weight_decay=weight_decay)
    
    return optimizer

In [3]:
def objective(data_list, num_epoch, model_name, trial):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    
    # 交差検証
    fold = KFold(
        n_splits=10, shuffle=True, random_state=0
    )

    valid_accs = []
    for fold_idx, (train_idx, valid_idx) in enumerate(fold.split(data_list)):
        
        if dataset.num_node_features == 0:
            num_feature = 1
        else:
            num_feature = dataset.num_node_features
        
        # set model
        if model_name =="GCN":
            model = GCN(hidden_channels=64, num_classes=4, num_node_feature=num_feature).to(device)
        elif model_name == "DGCNN":
            model = DGCNN_Model(num_features=num_feature, num_classes=4).to(device)
            
        optimizer = get_optimizer(trial, model)
        criterion = torch.nn.CrossEntropyLoss()
        
        # splot data
        train_loader = DataLoader(
            Subset(data_list, train_idx),
            shuffle=True,
            batch_size=50,
        )
        valid_loader = DataLoader(
            Subset(data_list, valid_idx),
            shuffle=False,
            batch_size=50,
        )
        for epoch_idx in range(num_epoch):
            # train
            cv_train(model, train_loader, device, criterion, optimizer, model_name)
            # valid
            valid_acc = cv_test(model, valid_loader, device, model_name)

        valid_accs.append(valid_acc)

#         print("fold {} : {}".format(fold_idx, valid_acc))
  
    # 返り値が最小となるようにハイパーパラメータチューニングが実行される
    return 1.0 - np.average(valid_accs)

In [4]:
from torch_geometric.utils import to_networkx
import networkx as nx
from tqdm import tqdm
def make_x(data):
    G = to_networkx(data)s
    new_x = torch.tensor(
        [[i] for i in list(dict(nx.degree(G)).values())],
        dtype=torch.float,
    )
    return new_x

In [5]:
df = pd.DataFrame(columns=["model", "dataset", "optimizer", "weight_decay", "adam_lr", "best_acc"])

cnt = 0
for dataset_name in ["MUTAG", "REDDIT-BINARY", "DD", "COLLAB"]:
    print(dataset_name)
    dataset = TUDataset(root='data/TUDataset', name=dataset_name)
    
    torch.manual_seed(12345)
    dataset = dataset.shuffle()
    
    # node attributeがない場合 degreeをnode attribute として使用
    if dataset[0].x is None:
        for d in dataset:
            d.x = make_x(d)
    
    datalist = []
    for d in tqdm(dataset):
        if d.x is None:
            d.x = make_x(d)
        datalist.append(d)
    
    for name in ["GCN", "DGCNN"]:
            # tuning
            study = optuna.create_study()
            f = partial(objective, datalist, 10, name)
            study.optimize(f, n_trials=10)
            study.trials_dataframe().to_csv(f"./paper_result/optuna/{name}_{dataset_name}.csv")

            # save best prams, best acc
            params = study.best_params
            params["model"] = name
            params["dataset"] = dataset_name
            params["best_acc"] = 1 - study.best_value
            df = df.append(pd.Series(params, name=cnt))
            cnt += 1

df.to_csv("paper_result/tuning_benchmark.csv")

100%|██████████| 188/188 [00:00<00:00, 21388.48it/s]
[I 2021-08-22 09:57:57,535] A new study created in memory with name: no-name-ffeac406-5ac9-41f7-85e1-1af52f775c44


MUTAG


[I 2021-08-22 09:58:01,032] Trial 0 finished with value: 0.3040935672514621 and parameters: {'optimizer': 'Adagrad', 'weight_decay': 1.4101547829633218e-06, 'adam_lr': 0.032921900613215614}. Best is trial 0 with value: 0.3040935672514621.
[I 2021-08-22 09:58:02,911] Trial 1 finished with value: 0.3345029239766081 and parameters: {'optimizer': 'Adagrad', 'weight_decay': 8.230625771835345e-08, 'adam_lr': 0.001912140444015722}. Best is trial 0 with value: 0.3040935672514621.
[I 2021-08-22 09:58:04,775] Trial 2 finished with value: 0.3666666666666666 and parameters: {'optimizer': 'Adagrad', 'weight_decay': 2.0657998284988788e-05, 'adam_lr': 0.06559534161441019}. Best is trial 0 with value: 0.3040935672514621.
[I 2021-08-22 09:58:06,638] Trial 3 finished with value: 0.3345029239766081 and parameters: {'optimizer': 'Adagrad', 'weight_decay': 1.117660764126913e-06, 'adam_lr': 5.785561750645676e-05}. Best is trial 0 with value: 0.3040935672514621.
[I 2021-08-22 09:58:08,502] Trial 4 finished w

REDDIT-BINARY


100%|██████████| 2000/2000 [00:03<00:00, 530.08it/s]
[I 2021-08-22 09:58:55,032] A new study created in memory with name: no-name-00eb5ac2-103c-49b5-b2c6-202d35468bf9
[I 2021-08-22 09:59:13,815] Trial 0 finished with value: 0.3045 and parameters: {'optimizer': 'Adam', 'weight_decay': 1.9579889180806796e-10, 'adam_lr': 0.0008257486513158593}. Best is trial 0 with value: 0.3045.
[I 2021-08-22 09:59:32,340] Trial 1 finished with value: 0.3075000000000001 and parameters: {'optimizer': 'Adam', 'weight_decay': 1.8816662289374288e-08, 'adam_lr': 0.07152849958450279}. Best is trial 0 with value: 0.3045.
[I 2021-08-22 09:59:50,139] Trial 2 finished with value: 0.3035 and parameters: {'optimizer': 'Adagrad', 'weight_decay': 1.4535660561720567e-09, 'adam_lr': 0.008201568390820502}. Best is trial 2 with value: 0.3035.
[I 2021-08-22 10:00:08,765] Trial 3 finished with value: 0.5 and parameters: {'optimizer': 'Adam', 'weight_decay': 1.8006151141949032e-10, 'adam_lr': 4.996610286532846e-05}. Best is 

DD


[I 2021-08-22 10:08:01,002] Trial 0 finished with value: 0.41336375488917876 and parameters: {'optimizer': 'Adam', 'weight_decay': 0.0004135882745992727, 'adam_lr': 0.0008790871698581789}. Best is trial 0 with value: 0.41336375488917876.
[I 2021-08-22 10:08:14,840] Trial 1 finished with value: 0.41336375488917876 and parameters: {'optimizer': 'Adagrad', 'weight_decay': 1.222179055025283e-06, 'adam_lr': 0.0005987681562132393}. Best is trial 0 with value: 0.41336375488917876.
[I 2021-08-22 10:08:29,156] Trial 2 finished with value: 0.399789946400116 and parameters: {'optimizer': 'Adam', 'weight_decay': 3.8748090704948254e-05, 'adam_lr': 0.0018281540020360044}. Best is trial 2 with value: 0.399789946400116.
[I 2021-08-22 10:08:43,081] Trial 3 finished with value: 0.40404172099087354 and parameters: {'optimizer': 'Adagrad', 'weight_decay': 1.4873013427080057e-09, 'adam_lr': 0.011678867219468151}. Best is trial 2 with value: 0.399789946400116.
[I 2021-08-22 10:08:57,547] Trial 4 finished wi

COLLAB


100%|██████████| 5000/5000 [00:37<00:00, 132.33it/s]
[I 2021-08-22 10:15:00,765] A new study created in memory with name: no-name-3d20100e-d40e-49c4-a5e9-43e32eaa7cf0
[I 2021-08-22 10:16:03,980] Trial 0 finished with value: 0.32219999999999993 and parameters: {'optimizer': 'Adagrad', 'weight_decay': 9.669829723306696e-09, 'adam_lr': 0.011800591304925293}. Best is trial 0 with value: 0.32219999999999993.
[I 2021-08-22 10:17:06,947] Trial 1 finished with value: 0.675 and parameters: {'optimizer': 'Adagrad', 'weight_decay': 6.611975978942618e-06, 'adam_lr': 0.000333221659513898}. Best is trial 0 with value: 0.32219999999999993.
[I 2021-08-22 10:18:12,107] Trial 2 finished with value: 0.6748000000000001 and parameters: {'optimizer': 'Adam', 'weight_decay': 4.430161193521281e-06, 'adam_lr': 2.254802593760215e-05}. Best is trial 0 with value: 0.32219999999999993.
[I 2021-08-22 10:19:15,366] Trial 3 finished with value: 0.3196000000000001 and parameters: {'optimizer': 'Adagrad', 'weight_decay

In [6]:
df

,model,dataset,optimizer,weight_decay,adam_lr,best_acc
0,GCN,MUTAG,Adam,1.216473e-06,0.003742,0.665497
1,DGCNN,MUTAG,Adagrad,2.166758e-10,0.008390,0.855848
